# Optimized Species Distribution Modeling (SDM)

This notebook contains a streamlined and modularized version of the SDM workflow. 
It includes steps for initialization, data preparation, model training, prediction, and export.

In [ ]:
# 1. Setup and Imports
import ee
import geemap
import pandas as pd
import numpy as np

# Initialize Earth Engine
try:
    ee.Initialize(project='cryptic-yen-457008-p4')
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project='cryptic-yen-457008-p4')

In [ ]:
# 2. Configuration
# Centralize all constants and paths here for easy management

CONFIG = {
    'BANDS': ['OrderST', 'aspect', 'elevation', 'slope', 'bio01', 'bio12'],
    'ASSETS': {
        'SOIL': "projects/cryptic-yen-457008-p4/assets/IsraelSoilTaxonomy",
        'MODEL_OUTPUT': 'projects/cryptic-yen-457008-p4/assets/avocado_final_model'
    },
    'VISUALIZATION': {
        'SUITABILITY': {"min": 0, "max": 1, "palette": ["ffffff", "cecece", "fcd163", "66a000", "204200"]},
        'DIFF': {"min": -0.3, "max": 0.3, "palette": ["d7191c", "ffffff", "2c7bb6"]}
    },
    'EXPORT': {
        'FOLDER': 'GEE_Exports',
        'SCALE': 1000
    }
}

In [ ]:
# 3. Helper Functions

def get_predictors():
    """Loads and preprocesses predictor variables."""
    # Topography
    terrain = ee.Algorithms.Terrain(ee.Image("USGS/SRTMGL1_003")).unmask()
    
    # Soil
    soil_fc = ee.FeatureCollection(CONFIG['ASSETS']['SOIL'])
    u_types = soil_fc.aggregate_array('OrderST').distinct().sort()
    soil_img = soil_fc.map(lambda f: f.set('Code', u_types.indexOf(f.get('OrderST')))) \
        .reduceToImage(['Code'], ee.Reducer.first()).rename('OrderST').unmask(-1)
    
    # Climate (Current)
    bio_curr = ee.Image("WORLDCLIM/V1/BIO").select(['bio01', 'bio12']).unmask()
    
    # Combine all
    return bio_curr.addBands([soil_img, terrain.select(['elevation', 'slope', 'aspect'])])

def prepare_training_data(presence_data, predictors, aoi, num_trees=250):
    """Generates pseudo-absences and prepares training data."""
    # Mark presence
    pres = presence_data.map(lambda f: f.set("PresAbs", 1))
    
    # Generate pseudo-absences
    abs_ = predictors.sample(region=aoi, scale=1000, numPixels=pres.size().add(200), geometries=True) \
        .randomColumn().sort('random').limit(pres.size()).map(lambda f: f.set("PresAbs", 0))
    
    # Merge and sample predictors
    training_data = predictors.select(CONFIG['BANDS']).sampleRegions(
        collection=pres.merge(abs_), 
        properties=["PresAbs"], 
        scale=1000, 
        tileScale=16
    )
    return training_data

def train_model(training_data, mode='MULTIPROBABILITY'):
    """Trains the Random Forest classifier."""
    classifier = ee.Classifier.smileRandomForest(250).train(training_data, "PresAbs", CONFIG['BANDS'])
    return classifier.setOutputMode(mode)

def get_future_climate(scenario, model='ACCESS1-0', year=2050):
    """Fetches and processes future climate data."""
    nex = ee.ImageCollection("NASA/NEX-GDDP") \
        .filter(ee.Filter.date(f'{year}-01-01', f'{year}-12-31')) \
        .filter(ee.Filter.eq('scenario', scenario)) \
        .filter(ee.Filter.eq('model', model))

    def convert(img):
        pr = img.select('pr').multiply(86400).rename('precip_mm')
        tm = img.select('tasmin').add(img.select('tasmax')).divide(2).subtract(273.15).rename('tmean_c')
        return img.addBands([pr, tm])

    nex_agg = nex.map(convert).map(lambda i: i.resample('bilinear').reproject('EPSG:4326', None, 1000))
    bio01 = nex_agg.select('tmean_c').mean().rename('bio01')
    bio12 = nex_agg.select('precip_mm').sum().rename('bio12')
    
    return bio01.addBands(bio12)

def export_image_to_drive(image, description, filename, region):
    """Creates and starts an export task."""
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=description,
        folder=CONFIG['EXPORT']['FOLDER'],
        fileNamePrefix=filename,
        scale=CONFIG['EXPORT']['SCALE'],
        region=region,
        maxPixels=1e13
    )
    task.start()
    print(f"Started export task: {description}")

In [ ]:
# 4. Main Execution Flow

# --- A. Prepare Data ---
print("Preparing data...")
predictors = get_predictors()
soil_fc = ee.FeatureCollection(CONFIG['ASSETS']['SOIL'])
aoi = soil_fc.geometry().bounds()

# NOTE: 'data' variable (presence points) needs to be defined or loaded here.
# Assuming it was loaded from a CSV or Asset in previous steps not fully visible in the snippet.
# Example placeholder:
# data = ee.FeatureCollection("users/your_username/avocado_points") 
# For now, I will comment out the training call to prevent errors if 'data' is missing.

# --- B. Train Model ---
# if 'data' in locals():
#     print("Training model...")
#     training_data = prepare_training_data(data, predictors, aoi)
#     rf_model = train_model(training_data)
# else:
#     print("WARNING: 'data' variable not found. Skipping training.")

# --- C. Future Predictions ---
# Assuming we have a trained model (rf_model). 
# If you are loading a saved model:
# rf_model = ee.Classifier.load(CONFIG['ASSETS']['MODEL_OUTPUT'])

# For demonstration, let's define the prediction logic wrapper
def predict_suitability(model, climate_stack, static_predictors):
    full_stack = climate_stack.addBands(static_predictors.select(['OrderST', 'elevation', 'slope', 'aspect']))
    return full_stack.select(CONFIG['BANDS']).classify(model).arrayGet([1])

# Example usage (commented out until model is ready):
# print("Predicting future scenarios...")
# future_climate_rcp45 = get_future_climate('rcp45')
# map_rcp45 = predict_suitability(rf_model, future_climate_rcp45, predictors)

# future_climate_rcp85 = get_future_climate('rcp85')
# map_rcp85 = predict_suitability(rf_model, future_climate_rcp85, predictors)

# diff_map = map_rcp85.subtract(map_rcp45)

In [ ]:
# 5. Visualization
Map = geemap.Map(layout={'height':'600px', 'width':'100%'})
Map.centerObject(aoi, 7)

# Add layers (Uncomment when maps are generated)
# Map.addLayer(map_rcp45.clip(aoi), CONFIG['VISUALIZATION']['SUITABILITY'], "Future Suitability 2050 (RCP 4.5)")
# Map.addLayer(map_rcp85.clip(aoi), CONFIG['VISUALIZATION']['SUITABILITY'], "Future Suitability 2050 (RCP 8.5)")
# Map.addLayer(diff_map.clip(aoi), CONFIG['VISUALIZATION']['DIFF'], "Difference (Red=Worse in 8.5)")

# Map.add_colorbar(CONFIG['VISUALIZATION']['SUITABILITY'], label="Suitability Probability", layer_name="Future Suitability")
# Map.add_colorbar(CONFIG['VISUALIZATION']['DIFF'], label="Change (Negative=Loss)", layer_name="Difference")

Map

In [ ]:
# 6. Exports
# export_image_to_drive(map_rcp45, 'export_rcp45', 'avocado_rcp45_2050', aoi)
# export_image_to_drive(map_rcp85, 'export_rcp85', 'avocado_rcp85_2050', aoi)